# R-Forge:

Le but c'est de récupérer la liste des paquets de R-Forge, ci dessous la procédure:

In [2]:

from bs4 import BeautifulSoup
import urllib2
#file = open('repoforge.csv', 'w')
for i in range(1, 95):
    url = urllib2.urlopen('https://r-forge.r-project.org/softwaremap/full_list.php?page='+str(i))
    html = url.read()
    soup = BeautifulSoup(html)
    for k in soup.find_all('span', property="doap:name"):
        nom = k.string      #ici on a les nom des packages
        lien = k.find_parent("a")
        hflien = lien.get("href")
    #file.write(nom.encode('utf-8') +","+ hflien.encode('utf-8') +"\n")

Jusqu'ici on récupère les noms des paquets avec les liens, il s'agit de 1880 paquets (pour les avoir dans un fichier csv il suffit de decommenter la 3eme et la dernière ligne). Ensuite pour récupérer le description file il faut extraire le 'vrai nom' depuis le liens d'avant (hflien) et le concatener à chaque fois avec le lien ou y a le descriptin file. les problèmes rencontrés ici c'est que dans le cas ou le paquet ne possède pas de description file on a un contenu html ou c'est noté erreur 404 sinon dans le cas ou le description file existe on a de text. Donc il faut filtrer le contenu.

In [ ]:
import  csv
import requests

#des = open('DfRf.txt', 'w')
file = csv.reader(open('repoforge.csv', 'rb'))   #je récupère mon fichier csv (nom,lien)

n=m=l=0

for row in file:
    if len(row)>0:
        url = row[1]
        x = url.rsplit("/", 2)
        nom = x[1]    #je récupère le 'vrai nom du paquet'
        content = requests.get("https://r-forge.r-project.org/scm/viewvc.php/*checkout*/pkg/DESCRIPTION?root=" + nom)
        if content.content.startswith('\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"'):
            n+=1
        else:
            if content.content.startswith('\n<?xml version="1.0" encoding="utf-8"?>'):
                m+=1
            else:
                #print >> des, content.content

On utiliser une autre approche pour avoir le description file avec un historique aussi c'est d'utiliser BeatifulSoup on suivant le lien du départ puis les liens qui mènent vers tous les descriptions files (pas seulement de la dernière version) cette procèdure est couteuse au niveau temporelle (ci dessous le code sans prendre en compte le fichier ou on a nom et le lien du paquet)

In [ ]:
from bs4 import BeautifulSoup
import urllib2

#file = open('extractALLDF.txt', 'w')
for i in range(1, 95):
    url = urllib2.urlopen('https://r-forge.r-project.org/softwaremap/full_list.php?page='+str(i))
    html = url.read()
    soup = BeautifulSoup(html)
    for k in soup.find_all('span', property="doap:name"):
        nom = k.string      #ici on a les nom des packages
        lien = k.find_parent("a")
        hflien = lien.get("href")
        rep = hflien.rsplit("/", 2)
        repo = rep[1]
        sitePack =["https://r-forge.r-project.org/scm/viewvc.php/pkg/?root=", repo]
        urlpack = "".join(sitePack)  #### jusque ici j'ai les lien vers les pkg ou y a DF
        lec = urllib2.urlopen(urlpack)
        res = lec.read()
        sp = BeautifulSoup(res)
        for item in sp.find_all('a', attrs={'name':'DESCRIPTION'}):
            lDesc = item.get("href")
            compDesc = ["https://r-forge.r-project.org", lDesc]
            compDesc2 = "".join(compDesc)
            dwnlec=urllib2.urlopen(compDesc2)
            rdDwnLec = dwnlec.read()
            sDown = BeautifulSoup(rdDwnLec)
            for d in sDown.find_all('a', text="download"):
                hrefDown = d.get("href")
                joinlinkDown = ["https://r-forge.r-project.org", hrefDown]
                linkDown = "".join(joinlinkDown)
                opLinkDown = urllib2.urlopen(linkDown)
                rdLinkDown = opLinkDown.read()
                print(rdLinkDown)